In [ ]:
import tensorflow as tf
from pyramda import compose, curry
import os
import numpy as np
import sys
from sklearn.model_selection import train_test_split
from visuazlizers.nb_graph_visualizer import show_graph, rename_nodes
from utils.curried_functions import tf_add, tf_cast, tf_multiply, filter_list
from loaders import image_batch_loader, load_model_pb
from siamese import compute_loss, get_anchor_positive_mask, get_negative_mask, train_siamese_model
from utils.metrics import cosine_distance

### Batch

In [ ]:
classes_to_labels = compose(
    list,
    range,
    len,
)

### Model

In [ ]:
import sys
sys.path.append("..")
from input.models.deep_sort_cnn.freeze_model import _preprocess, _network_factory

def create_deep_sort_cnn_graph():
    input_var = tf.placeholder(tf.uint8, (None, 128, 64, 3), name="images")
    image_var = tf.map_fn(
        lambda x: _preprocess(x), tf.cast(input_var, tf.float32),
        back_prop=False
    )

    factory_fn = _network_factory()
    features, _ = factory_fn(image_var, reuse=None)
    features = tf.identity(features, name="features")
    
    return input_var, features

### Training

In [ ]:
tf.reset_default_graph()

source_path = '../input/mars/bbox_train/'
dirs = compose(
    filter_list(['.DS_Store'], False),
    os.listdir,
)(source_path)

train_dirs, val_dirs = train_test_split(dirs, test_size=0.2)
train_labels = classes_to_labels(train_dirs)

inputs, outputs, _ = load_model_pb(
    '../input/models/deep_sort_cnn/mars-small128.pb', 
    input_name="images", 
    output_name="features", 
    graph_creator=create_deep_sort_cnn_graph,
)

session = tf.Session()

train_siamese_model(
    session=session,
    model=[inputs, outputs],
    source_path=source_path,
    dirs=(train_dirs[0:5], val_dirs),
    train_labels=train_labels[0:5],
    metric=cosine_distance,
    optimizer=tf.train.AdamOptimizer(learning_rate=0.00001),
    batch_loader=image_batch_loader((128, 64, 3)),
    num_iter=2,
)